## **Introdução**

Neste relatório, realizamos uma análise de dados utilizando um painel interativo desenvolvido com **Streamlit** e uma **planilha Excel**. O objetivo é monitorar padrões de incêndios no Brasil durante os três primeiros trimestres de 2024, com foco na **identificação de áreas críticas** e no **suporte à tomada de decisões estratégicas** para a prevenção e controle de incêndios florestais.

O **dataset utilizado** está disponível no Kaggle e pode ser acessado através do link:  
[Fire Watch Brazil 2024 - Kaggle](https://www.kaggle.com/datasets/mayaravalliero/fire-watch-brazil-2024/data?select=Dataset_FireWatch_Brazil_Q3_2024.csv).

Este conjunto de dados oferece informações detalhadas sobre:
- **Dias sem chuva**: Número médio de dias consecutivos sem chuva em cada localidade.
- **Precipitação**: Média de precipitação em milímetros (mm) ao longo do período.
- **Risco de Fogo**: Percentual médio que indica a probabilidade de ocorrência de incêndios florestais.
- **FRP (Fire Radiative Power)**: Potência Radiativa do Fogo, que mede a energia liberada pelos incêndios florestais e é um indicativo da intensidade dos eventos.

A relevância deste estudo reside em consolidar e comparar a eficácia de duas formas de apresentação dos resultados:
1. **Painel Interativo**: Permite uma exploração dinâmica e personalizada dos dados, oferecendo insights em tempo real.
2. **Planilha Estática**: Oferece uma visão rápida e organizada para consultas mais simples.

Através dessa comparação, o estudo busca mostrar como diferentes abordagens podem atender às necessidades de análise e comunicação de dados, auxiliando organizações na **prevenção de incêndios florestais** e na **tomada de decisões estratégicas**.


## **Bibliotecas Necessárias**

Neste projeto, foram utilizadas diversas bibliotecas para **manipulação de dados**, **visualização gráfica**, **análises estatísticas** e a **criação do painel interativo** no **Streamlit**. Abaixo está uma explicação do uso de cada uma delas no contexto deste trabalho.

### 1. **NumPy** (`numpy`)
- Utilizada para **operações matemáticas** e criação de **arrays** que facilitam cálculos numéricos durante a análise dos dados climáticos.
- Foi empregada principalmente para **calcular médias e somatórios** necessários na substituição de valores ausentes (por exemplo, preenchimento de `avg_frp` com a média dos valores existentes).

### 2. **Pandas** (`pandas`)
- Essencial para a **manipulação e limpeza de dados**, sendo a base para carregar, filtrar e transformar os dados climáticos e de risco de incêndio.
- Permite a **filtragem por estado e município** no painel interativo e a exibição dos dados em uma tabela no Streamlit.

### 3. **Matplotlib** (`matplotlib.pyplot`)
- Utilizada para **construção de gráficos** no painel, incluindo gráficos de linha para mostrar a variação da precipitação, risco de fogo e FRP ao longo do tempo.
- Fornece uma **interface personalizável** para ajustar o layout e a aparência dos gráficos.

### 4. **Seaborn** (`seaborn`)
- Uma extensão do Matplotlib, utilizada para **gráficos estatísticos mais sofisticados e elegantes**.
- No projeto, foi usada para gerar gráficos de **linha** que mostram a evolução dos indicadores climáticos e de incêndios, com configurações de estilo mais refinadas.

### 5. **Statsmodels** (`statsmodels.api`)
- Biblioteca utilizada para **modelagem estatística**. No contexto do projeto, permite realizar análises mais complexas, como a criação de **modelos de regressão linear** para entender a relação entre variáveis climáticas e o risco de incêndio.

### 6. **SciPy** (`scipy.stats`)
- Fornece ferramentas estatísticas adicionais. No projeto, a função `pearsonr` foi utilizada para **calcular a correlação** entre diferentes variáveis, como a relação entre **precipitação e risco de incêndio**.

### 7. **Unidecode** (`unidecode`)
- Utilizada para **normalização de textos**, removendo acentos e caracteres especiais nos nomes de municípios e estados.
- Garante que os **filtros no Streamlit** funcionem corretamente, evitando problemas ao comparar strings que podem conter acentos ou outros caracteres não padronizados.

### 8. **Streamlit** (`streamlit`)
- A principal ferramenta para a **criação do painel interativo**. Permite que os usuários filtrem dados por **estado e município**, visualizem tabelas e gráficos, e explorem as informações de forma dinâmica e intuitiva.
- Streamlit facilita a **interação em tempo real** com os dados, permitindo a criação de relatórios visuais diretamente no navegador, sem necessidade de interfaces complexas.

---

Com essas bibliotecas, o projeto combina **análises estatísticas** com **visualizações interativas**, fornecendo uma plataforma poderosa para explorar padrões climáticos e de risco de incêndio em diferentes regiões do Brasil.


In [156]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import pearsonr
from unidecode import unidecode
import streamlit as st

## **Processo de ETL: Extração, Transformação e Carga**

O processo de ETL (Extração, Transformação e Carga) é essencial para preparar dados brutos para análise e visualização.

### **1. Extração**
Consiste em carregar os dados a partir de fontes, como arquivos CSV. Nesta etapa, é importante garantir que todos os dados relevantes sejam obtidos.

### **2. Transformação**
Os dados brutos são limpos e organizados para garantir consistência. Esta etapa pode incluir:
- Tratamento de valores ausentes.
- Modificação de tipos de dados.
- Consolidação de datasets (ex.: unir dados trimestrais).

### **3. Carga**
Nesta fase, os dados tratados são exportados e disponibilizados para análise, seja em arquivos (como Excel) ou em um painel interativo (como o Streamlit). A carga é crucial para garantir que os dados estejam prontos para gerar insights e facilitar a exploração pelo usuário.

O ETL assegura que os dados estejam limpos, organizados e prontos para uso em relatórios e visualizações.


## **Carregando os Dados**

In [157]:
df1 = pd.read_csv('/content/Dataset_FireWatch_Brazil_Q1_2024.csv')
df2 = pd.read_csv('/content/Dataset_FireWatch_Brazil_Q2_2024.csv')
df3 = pd.read_csv('/content/Dataset_FireWatch_Brazil_Q3_2024.csv')

## **Concatenando os Dados**

**Concatenação dos Dados**:
   - Todos os registros dos três trimestres foram concatenados em um único DataFrame para facilitar a análise.

In [158]:
df1.info(), df2.info(), df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43753 entries, 0 to 43752
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   data                       43753 non-null  object 
 1   municipio                  43753 non-null  object 
 2   estado                     43753 non-null  object 
 3   bioma                      43752 non-null  object 
 4   avg_numero_dias_sem_chuva  43753 non-null  float64
 5   avg_precipitacao           43753 non-null  float64
 6   avg_risco_fogo             43753 non-null  float64
 7   avg_frp                    43753 non-null  float64
dtypes: float64(4), object(4)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48979 entries, 0 to 48978
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   data                       48979 non-null  object 
 1  

(None, None, None)

In [159]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df.head()

,data,municipio,estado,bioma,avg_numero_dias_sem_chuva,avg_precipitacao,avg_risco_fogo,avg_frp
0,2024-01-01,VALE DO ANARI,RONDÔNIA,Amazônia,0.0,1698.00,0.0,13.00
1,2024-01-01,SÃO GABRIEL DA CACHOEIRA,AMAZONAS,Amazônia,0.2,206.80,0.0,81.40
2,2024-01-01,MANAQUIRI,AMAZONAS,Amazônia,0.0,60.00,0.0,27.00
3,2024-01-01,JUTAÍ,AMAZONAS,Amazônia,2.0,776.00,0.0,211.00
4,2024-01-01,AUTAZES,AMAZONAS,Amazônia,0.0,2.22,0.0,97.67


## **Conhecendo os Dados**

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162093 entries, 0 to 162092
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   data                       162093 non-null  object 
 1   municipio                  162093 non-null  object 
 2   estado                     162093 non-null  object 
 3   bioma                      162092 non-null  object 
 4   avg_numero_dias_sem_chuva  162093 non-null  float64
 5   avg_precipitacao           162093 non-null  float64
 6   avg_risco_fogo             162093 non-null  float64
 7   avg_frp                    162089 non-null  float64
dtypes: float64(4), object(4)
memory usage: 9.9+ MB


In [161]:
df.shape

(162093, 8)

In [162]:
# Verificando a porcentagem de nulos em cada coluna
null_percentage = df.isnull().mean() * 100
print(null_percentage[null_percentage > 0])

bioma      0.000617
avg_frp    0.002468
dtype: float64


## **Tratamento dos Dados**

Foram aplicadas as seguintes transformações nos dados:

1. **Tratamento de Valores Ausentes**:
   - **Preenchimento de `bioma` ausente**:
     - Nos casos em que o município era `'LAGOA DOS PATOS'`, o valor nulo foi substituído por `'Pampa'`:
     - Para os demais valores ausentes na coluna `bioma`, foi preenchido com **"Não identificado"**:

   - **Preenchimento de `avg_frp`**:  
     - Os valores ausentes foram preenchidos com a **média** dos valores existentes:

2. **Normalização de Texto**:
   - Removemos acentos e convertendo textos para **minúsculas**:

3. **Conversão de Tipos de Dados**:
   - A coluna `data` foi convertida para o formato **datetime**:

---

Após essas transformações, os dados estão limpos e preparados para análise. Essas etapas garantem a consistência e a integridade das informações, facilitando a exploração dos dados através de gráficos e relatórios.


In [163]:
# Verificar quais linhas possuem nulos em cada coluna
nulos_bioma = df[df['bioma'].isnull()]
nulos_avg_frp = df[df['avg_frp'].isnull()]

In [164]:
print("\nLinhas com nulos na coluna 'bioma':")
nulos_bioma


Linhas com nulos na coluna 'bioma':


,data,municipio,estado,bioma,avg_numero_dias_sem_chuva,avg_precipitacao,avg_risco_fogo,avg_frp
19130,2024-01-30,LAGOA DOS PATOS,RIO GRANDE DO SUL,NaN,0.0,0.0,0.0,0.0


In [165]:
# Visualizando os Valores Únicos de cada Bioma, para realizar a substituição
df['bioma'].unique()

array(['Amazônia', 'Cerrado', 'Caatinga', 'Mata Atlântica', 'Pampa',
       'Pantanal', nan], dtype=object)

In [166]:
# Substituir o valor nulo na coluna 'bioma' para 'Pampa'
df.loc[(df['municipio'] == 'LAGOA DOS PATOS') & (df['bioma'].isnull()), 'bioma'] = 'Pampa'
df.loc[(df['bioma'] == 'Pampa') & (df['municipio'] == 'LAGOA DOS PATOS')]

,data,municipio,estado,bioma,avg_numero_dias_sem_chuva,avg_precipitacao,avg_risco_fogo,avg_frp
19130,2024-01-30,LAGOA DOS PATOS,RIO GRANDE DO SUL,Pampa,0.0,0.0,0.0,0.0


In [167]:
print("\nLinhas com nulos na coluna 'avg_frp':")
nulos_avg_frp


Linhas com nulos na coluna 'avg_frp':


,data,municipio,estado,bioma,avg_numero_dias_sem_chuva,avg_precipitacao,avg_risco_fogo,avg_frp
160095,2024-09-15,JANUÁRIA,MINAS GERAIS,Cerrado,0.0,0.0,0.0,NaN
160658,2024-09-15,OLÍMPIO NORONHA,MINAS GERAIS,Mata Atlântica,0.0,0.0,0.0,NaN
160667,2024-09-15,LAJEDÃO,BAHIA,Mata Atlântica,0.0,0.0,0.0,NaN
162071,2024-09-16,GRAÇA ARANHA,MARANHÃO,Cerrado,0.0,0.0,0.0,NaN


In [168]:
# Preenchendo os valores Nulos na coluna avg_frp com a mediana
df['avg_frp'].fillna(df['avg_frp'].median(), inplace=True)

<ipython-input-168-186e8f341e9a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['avg_frp'].fillna(df['avg_frp'].median(), inplace=True)


In [169]:
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d', errors='coerce')

# Verificar o resultado
print(df['data'].head())

0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: data, dtype: datetime64[ns]


In [170]:
# Convertendo a coluna de data para datetime
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

In [171]:
# Criando uma nova coluna com o agrupamento por mês
df['mes_ano'] = df['data'].dt.to_period('M')

In [172]:
# Função para remover acentuação
def remover_acentuacao(df, colunas):
    for coluna in colunas:
        df[coluna] = df[coluna].apply(lambda x: unidecode(str(x)) if pd.notnull(x) else x)
    return df

In [173]:
# Definir as colunas que você quer remover a acentuação
colunas_para_tratar = ['municipio', 'estado', 'bioma']

In [174]:
# Aplicar a função no DataFrame
df = remover_acentuacao(df, colunas_para_tratar)
df.head()

,data,municipio,estado,bioma,avg_numero_dias_sem_chuva,avg_precipitacao,avg_risco_fogo,avg_frp,mes_ano
0,2024-01-01,VALE DO ANARI,RONDONIA,Amazonia,0.0,1698.00,0.0,13.00,2024-01
1,2024-01-01,SAO GABRIEL DA CACHOEIRA,AMAZONAS,Amazonia,0.2,206.80,0.0,81.40,2024-01
2,2024-01-01,MANAQUIRI,AMAZONAS,Amazonia,0.0,60.00,0.0,27.00,2024-01
3,2024-01-01,JUTAI,AMAZONAS,Amazonia,2.0,776.00,0.0,211.00,2024-01
4,2024-01-01,AUTAZES,AMAZONAS,Amazonia,0.0,2.22,0.0,97.67,2024-01


## **Exportação de Dados para Excel**

**Preparação dos Dados para Exportação**:
   - Após a transformação, os dados foram exportados para uma planilha Excel organizada.

In [175]:
# Exportar para Excel
df.to_excel('Dataset_FireWatch_Consolidado.xlsx', index=False)

## **Análise Exploratória no Streamlit**

Esta seção apresenta um painel interativo desenvolvido com **Streamlit** para explorar dados climáticos e de risco de incêndios em diferentes municípios da Amazônia. O painel oferece filtros por **estado** e **município**, além de gráficos interativos para visualização de padrões ao longo do tempo.

---

### **Descrição do Painel**

- **Data**: Data da observação.
- **Município**: Nome do município onde os dados foram coletados.
- **Estado**: Estado brasileiro ao qual o município pertence.
- **Bioma**: Bioma da região analisada.
- **avg_numero_dias_sem_chuva**: Média do número de dias consecutivos sem chuva até a data específica.
- **avg_precipitacao**: Precipitação média em milímetros (mm) no período analisado.
- **avg_risco_fogo**: Percentual médio de risco de incêndios florestais.
- **avg_frp**: FRP (Fire Radiative Power) – Potência Radiativa do Fogo, que mede a energia liberada pelos incêndios florestais.

---

### **Funcionalidades do Painel**

1. **Seleção de Estado e Município**:
   - O usuário pode selecionar um **estado** da lista disponível.
   - Após selecionar o estado, são exibidos os **municípios** correspondentes para uma escolha mais específica.

2. **Exibição dos Dados Filtrados**:
   - Os dados filtrados são exibidos em uma tabela centralizada, com rolagem interna para facilitar a navegação.

3. **Gráficos Interativos**:
   - **Precipitação Média ao Longo do Tempo**: Visualiza a variação da quantidade de chuva (em mm) ao longo das datas observadas.
   - **Risco Médio de Fogo**: Apresenta a probabilidade de incêndios florestais, variando de 0 a 100%.
   - **FRP Médio ao Longo do Tempo**: Mede a intensidade dos incêndios florestais, com valores mais altos indicando incêndios mais intensos.

---

### **Configuração Visual**

O painel adota um layout com três colunas para otimizar a exibição dos gráficos, centralizando as visualizações. As configurações de estilo utilizam a biblioteca **Seaborn** para gráficos elegantes e informativos. Os gráficos exibem informações importantes sobre:
- Variação de precipitação.
- Mudança no risco de incêndio.
- Potência dos incêndios ao longo do tempo.

---

### **Conclusão**

Este painel permite uma análise intuitiva e interativa dos dados climáticos e de risco de incêndios. A combinação de filtros e gráficos facilita a exploração em tempo real, auxiliando na tomada de decisões e na identificação de padrões relevantes. A interface do **Streamlit** oferece


In [ ]:
# Streamlit: Título e descrição do painel
st.title("Painel de Análise Climática e Risco de Fogo")

# Seção de documentação
with st.expander("📄 Explicação e Documentação"):
    st.write("""
    **Este painel apresenta uma análise climática e de risco de fogo em diferentes municípios da Amazônia.**

    - **Data**: Representa a data da observação.
    - **Município**: Nome do município onde os dados foram coletados.
    - **Estado**: Estado brasileiro a que o município pertence.
    - **Bioma**: Indica o bioma da região.
    - **avg_numero_dias_sem_chuva**: Média do número de dias consecutivos sem chuva até a data especificada.
    - **avg_precipitacao**: Precipitação média em milímetros (mm) no período analisado.
    - **avg_risco_fogo**: Percentual médio de risco de incêndios florestais.
    - **avg_frp**: FRP (Fire Radiative Power), ou Potência Radiativa do Fogo, que mede a energia liberada pelos incêndios florestais.

    **Gráficos**:

    - O gráfico de "Precipitação Média" mostra como a quantidade de chuva (em mm) variou ao longo do tempo.
    - O gráfico de "Risco Médio de Fogo" apresenta a probabilidade de incêndios florestais, variando de 0 a 100%.
    - O gráfico de "FRP Médio" mede a potência dos incêndios, com valores mais altos indicando incêndios mais intensos.
    """)

# Filtro por estado e município
estados = df['estado'].unique()
estado_selecionado = st.selectbox("Selecione o estado", estados)

municipios = df[df['estado'] == estado_selecionado]['municipio'].unique()
municipio_selecionado = st.selectbox("Selecione o município", municipios)

# Filtrar os dados
df_filtrado = df[(df['estado'] == estado_selecionado) & (df['municipio'] == municipio_selecionado)]

st.write(f"Dados para {municipio_selecionado} - {estado_selecionado}")

# Centralizar a tabela com HTML e adicionar rolagem interna
st.markdown(
    """
    <div style="display: flex; justify-content: center;">
        <div style="overflow-x: auto; height: 400px;">
            {}
        </div>
    </div>
    """.format(
        df_filtrado.to_html(classes='dataframe', index=False, justify='center', escape=False)
    ),
    unsafe_allow_html=True
)

# Configurações visuais do Seaborn
sns.set(style="whitegrid", palette="muted")

# Layout de colunas para melhor utilização do espaço horizontal
col1, col2, col3 = st.columns([1, 6, 1])

with col2:
    # Visual: Precipitação Média ao longo do tempo
    fig, ax = plt.subplots(figsize=(12, 6))  # Aumentando o tamanho do gráfico
    sns.lineplot(x='data', y='avg_precipitacao', data=df_filtrado, ax=ax, marker='o', color='b', linewidth=2)
    ax.set_title('Precipitação Média ao Longo do Tempo', fontsize=16, fontweight='bold')
    ax.set_xlabel('Data', fontsize=12)
    ax.set_ylabel('Precipitação (mm)', fontsize=12)
    plt.xticks(rotation=45)
    sns.despine()
    st.pyplot(fig)

    # Visual: Risco Médio de Fogo
    fig, ax = plt.subplots(figsize=(12, 6))  # Aumentando o tamanho do gráfico
    sns.lineplot(x='data', y='avg_risco_fogo', data=df_filtrado, marker='o', ax=ax, color='r', linewidth=2)
    ax.set_title('Risco Médio de Fogo ao Longo do Tempo', fontsize=16, fontweight='bold')
    ax.set_xlabel('Data', fontsize=12)
    ax.set_ylabel('Risco de Fogo (%)', fontsize=12)
    plt.xticks(rotation=45)
    sns.despine()
    st.pyplot(fig)

    # Visual: FRP médio ao longo do tempo
    fig, ax = plt.subplots(figsize=(12, 6))  # Aumentando o tamanho do gráfico
    sns.lineplot(x='data', y='avg_frp', data=df_filtrado, marker='o', ax=ax, color='g', linewidth=2)
    ax.set_title('FRP Médio ao Longo do Tempo', fontsize=16, fontweight='bold')
    ax.set_xlabel('Data', fontsize=12)
    ax.set_ylabel('FRP Médio', fontsize=12)
    plt.xticks(rotation=45)
    sns.despine()
    st.pyplot(fig)

## Conclusão

A análise realizada permitiu explorar padrões de incêndio por meio de diferentes abordagens. A comparação entre a planilha Excel e o painel no Streamlit demonstrou que:

- **Planilha**: Ótima para consultas estáticas e análises simples.
- **Painel Interativo**: Fornece uma experiência mais rica e interativa, permitindo exploração em tempo real.

Com isso, a escolha entre os dois formatos depende do público-alvo e do contexto da análise. Ferramentas como o Streamlit se mostram poderosas para cenários que exigem maior interação e rapidez na tomada de decisões.
